In [1]:
import os
import azureml.core
from azureml.core import Workspace, Experiment, Datastore
from azureml.widgets import RunDetails

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.3.0


In [2]:
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep

print("Pipeline SDK-specific imports completed")

Pipeline SDK-specific imports completed


In [3]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

WARNING - Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


testworkspace
maytest
centralus
35f16a99-532a-4a47-9e93-00305f6c40f2


In [4]:
from azureml.core.compute import AmlCompute
aml_compute = AmlCompute(ws, "amlcomp")
print(aml_compute.get_status().serialize())

{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 1, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-05-14T22:55:48.762000+00:00', 'errors': None, 'creationTime': '2020-01-28T06:09:33.862975+00:00', 'modifiedTime': '2020-04-23T23:59:31.791376+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 16, 'nodeIdleTimeBeforeScaleDown': 'PT600S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D14_V2'}


In [5]:
# Use a RunConfiguration to specify some additional requirements for this step.
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.container_registry import ContainerRegistry

# create a new runconfig object
run_config = RunConfiguration()

# enable Docker 
run_config.environment.docker.enabled = True

# pointing to the default container registry in the workspace
image_registry_details = ContainerRegistry()
image_registry_details.address = "testworkspacc2c59dc5.azurecr.io"
run_config.environment.docker.base_image_registry = image_registry_details

# this is an image in the image_registry
image_name = 'azureml/odbcimage:latest' # should look like 'azureml/{image_name}:{tag}'
run_config.environment.docker.base_image = image_name

# use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_config.environment.python.user_managed_dependencies = True

# specify CondaDependencies obj
run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['pyodbc'])

# For this step, we use a source_directory that contains our python script that we want to run
source_directory = './script' # name of the directory containing the python script we wish to run 
print('Source directory for the step is {}.'.format(os.path.realpath(source_directory)))

sql_demo_step = PythonScriptStep(name="sql_step",
                         script_name="pyodbc_pipeline_step.py", 
                         compute_target=aml_compute, 
                         source_directory=source_directory,
                         runconfig=run_config)

print("sql_demo_step created")

Source directory for the step is C:\Users\SIHHU\Desktop\write to sql\script.
sql_demo_step created


In [6]:
#build and validate pipeline
steps=[sql_demo_step]
pipeline1 = Pipeline(workspace=ws, steps=steps)
print ("Pipeline is built, now validating...")
pipeline1.validate()
print("Pipeline validation complete")

Pipeline is built, now validating...
Step sql_step is ready to be created [a500fc9e]
Pipeline validation complete


In [7]:
# Submit for execution
pipeline_run1 = Experiment(ws, 'PYODBC_DEMO').submit(pipeline1, regenerate_outputs=False)

Created step sql_step [a500fc9e][6f8f1364-7461-4599-8b35-1e9f52b5375c], (This step will run and generate new outputs)
Submitted PipelineRun b1a7fc95-ab8e-4b28-b18c-05f0191addbb
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/PYODBC_DEMO/runs/b1a7fc95-ab8e-4b28-b18c-05f0191addbb?wsid=/subscriptions/35f16a99-532a-4a47-9e93-00305f6c40f2/resourcegroups/maytest/workspaces/testworkspace
